In [1]:
!pip install transformers

     |████████████████████████████████| 1.3MB 8.7MB/s 
     |████████████████████████████████| 2.9MB 31.2MB/s 
     |████████████████████████████████| 1.1MB 42.9MB/s 
     |████████████████████████████████| 890kB 52.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=e5d9048e7dcf305e7c9f7acc0d9c90fe4674ec7a4821a7c4ba4121496310486b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
# imports
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelWithLMHead
from matplotlib import pyplot as plt

from google.colab import files
files.upload()  # load dataGeneration.py
#files.upload()  # load waypointNet.py
from dataGeneration import generateTestBatch
#from waypointNet import WaypointNet, listToTensor

Saving dataGeneration.py to dataGeneration (1).py


In [3]:
# import the pretrained models
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelWithLMHead.from_pretrained("bert-base-uncased")

# helper for padding 2D lists into tensors
def listToTensor(list):
  maxLen = max(len(l) for l in list)
  tensor = torch.tensor([l + (maxLen - len(l)) * [0] for l in list])
  return tensor


/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# investigating BERT model output size
baseModel = model.bert

sampleBatch = ["go forward three meters", "turn right", "go right 1 meter"]
enc = tokenizer.batch_encode_plus(sampleBatch)
X = listToTensor(enc["input_ids"])
attn = listToTensor(enc["attention_mask"])

out = baseModel(X, attn)
print(out[0].shape)
print("Batch size, length of tokenized text, model output size\n")

lstm = nn.LSTM(input_size=768, hidden_size=100, num_layers = 2, dropout=0.05)
out = lstm(out[0].permute(1,0,2))[0].permute(1,0,2)
print(out.shape)

torch.Size([3, 6, 768])
Batch size, length of tokenized text, model output size

torch.Size([3, 6, 100])


In [6]:
# custom head for BERT
# credit to Martin Zablocki: https://zablo.net/blog/post/custom-classifier-on-bert-model-guide-polemo2-sentiment-analysis/
class WaypointNet(nn.Module):
  def __init__(self, base_model, base_output_size=768, hidden_size = 100, dropout=0.05):
    super().__init__()
    self.base_model = base_model
    
    self.lstm = nn.LSTM(input_size=base_output_size, hidden_size=hidden_size,
                        num_layers=2, dropout=dropout)
    self.linear = nn.Linear(hidden_size, 3)

  def forward(self, input, *args):
    X, attention_mask = input
    hidden_states = self.base_model(X, attention_mask = attention_mask)[0]
    lstm_out = self.lstm(hidden_states.permute(1,0,2))[0].permute(1,0,2)
    return self.linear(lstm_out[:, -1, :])


In [ ]:
# custom head for BERT
# credit to Martin Zablocki: https://zablo.net/blog/post/custom-classifier-on-bert-model-guide-polemo2-sentiment-analysis/
# credit to Michael Sugimura: https://towardsdatascience.com/bert-classifier-just-another-pytorch-model-881b3cf05784
class WaypointNet2(nn.Module):
  def __init__(self, base_model, base_output_size=768, hidden_size = 100, dropout=0.05):
    super().__init__()
    self.bert = base_model
    self.linear = nn.Linear(hidden_size, 3)
    self.dropout = nn.Dropout(dropout)

  def forward(self, input, *args):
    X, attention_mask = input
    _, pooled_output = hidden_states = self.bert(X, attention_mask = attention_mask)
    pooled_output = self.dropout(pooled_output)
    output = self.linear(pooled_output)
    return output
    


In [ ]:
# custom head for BERT
# credit to Martin Zablocki: https://zablo.net/blog/post/custom-classifier-on-bert-model-guide-polemo2-sentiment-analysis/
# credit to Michael Sugimura: https://towardsdatascience.com/bert-classifier-just-another-pytorch-model-881b3cf05784
class WaypointNet3(nn.Module):
  def __init__(self, base_model, base_output_size=768, hidden_size = 100, dropout=0.05):
    super().__init__()
    self.bert = base_model
    self.linear = nn.Linear(hidden_size, 3)
    self.dropout = nn.Dropout(dropout)

  def forward(self, input, *args):
    X, attention_mask = input
    _, pooled_output = hidden_states = self.bert(X, attention_mask = attention_mask)
    pooled_output = self.dropout(pooled_output)
    output = self.linear(pooled_output)
    return output

In [ ]:
# training model
if torch.cuda.is_available():  
  device = torch.device("cuda:0") 
else:  
  device = torch.device("cpu")
print(device)

net = WaypointNet(baseModel)
net = net.to(device=device)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
objectiveValues = []
running_loss = 0

# each epoch is a newly generated batch of test data
for epoch in range(5000):
  text, labels = generateTestBatch(1000)
  labels = torch.tensor(labels, dtype = torch.float)
  enc = tokenizer.batch_encode_plus(text)
  X = listToTensor(enc["input_ids"])
  attn = listToTensor(enc["attention_mask"])

  X = X.to(device)
  attn = attn.to(device)
  labels = labels.to(device)

  # zero the parameter gradients
  optimizer.zero_grad()

  # forward + backward + optimize
  outputs = net((X, attn))
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()

  # print statistics
  running_loss += loss.item()
  if ((epoch + 1) % 100 == 0):
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / 100))
    running_loss = 0
  objectiveValues.append(loss.item())

plt.plot(objectiveValues)

cuda:0
[100] loss: 3.520
[200] loss: 2.276
[300] loss: 1.248
[400] loss: 0.418
[500] loss: 0.141
[600] loss: 0.067
[700] loss: 0.039
[800] loss: 0.030
[900] loss: 0.022
[1000] loss: 0.018
[1100] loss: 0.015
[1200] loss: 0.014
[1300] loss: 0.013
[1400] loss: 0.011
[1500] loss: 0.010


Here is some code that compares two neural networks by training them simultaneously. WARNING: SLOW

In [ ]:
# training model
if torch.cuda.is_available():  
  device = torch.device("cuda:0") 
else:  
  device = torch.device("cpu")
print(device)

net = WaypointNet(baseModel)
net = net.to(device=device)
net2 = WaypointNet2(baseModel)
net2 = net2.to(device=device)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer2 = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
objectiveValues = []
objectiveValues2 = []
running_loss = 0
running_loss2 = 0

# each epoch is a newly generated batch of test data
for epoch in range(5000):
  text, labels = generateTestBatch(1000)
  labels = torch.tensor(labels, dtype = torch.float)
  enc = tokenizer.batch_encode_plus(text)
  X = listToTensor(enc["input_ids"])
  attn = listToTensor(enc["attention_mask"])

  X = X.to(device)
  attn = attn.to(device)
  labels = labels.to(device)

  # zero the parameter gradients
  optimizer.zero_grad()
  optimizer2.zero_grad()

  # forward + backward + optimize
  outputs = net((X, attn))
  outputs2 = net2((X, attn))
  loss = criterion(outputs, labels)
  loss.backward()
  loss2 = criterion(outputs2, labels)
  loss2.backward()
  optimizer.step()
  optimizer2.step()

  # print statistics
  running_loss += loss.item()
  running_loss2 += loss2.item()
  if ((epoch + 1) % 100 == 0):
    print('[%d] loss for model 1: %.3f' % (epoch + 1, running_loss / 100))
    print('[%d] loss for model 2: %.3f' % (epoch + 1, running_loss2 / 100))
    running_loss = 0
    running_loss2 = 0
  objectiveValues.append(loss.item())
  objectiveValues2.append(loss2.item())

plt.plot(objectiveValues)
plt.plot(objectiveValues2)
torch.save(net.state_dict(), "waypointBERT.pth")
files.download("waypointBERT.pth")
torch.save(net2.state_dict(), "waypointBERT2.pth")
files.download("waypointBERT2.pth")

In [ ]:
# save model
torch.save(net.state_dict(), "waypointBERT.pth")
files.download("waypointBERT.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# test the output of the network
with torch.no_grad():
  text = ["go 1 meter forward",
      "then turn left",
      "followed by moving 2 and a half feet up and to the right"]
  enc = tokenizer.batch_encode_plus(text)
  X = listToTensor(enc["input_ids"])
  attn = listToTensor(enc["attention_mask"])
  X = X.to(device)
  attn = attn.to(device)

  output = net((X, attn))

  print(text)
  print(output)

['go 1 meter forward', 'then turn left', 'followed by moving 2 and a half feet up and to the right']
tensor([[ 1.0603, -0.0302, -0.0185],
        [ 0.0031,  0.0345,  1.5826],
        [ 0.6163, -0.5090, -0.8517]], device='cuda:0')


In [ ]:
# clear memory
import gc
del X
del attn
del labels
del net

gc.collect()


256